In [ ]:
# Import the required libraries and dependencies
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import sqlalchemy

# Add viz libs
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import hvplot.pandas
import altair as alt 
from bokeh.plotting import figure, show
import folium
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
from PIL import Image
from plotnine import ggplot, aes, geom_line

## Clean data

In [ ]:
# Force clean vars
force_clean_large_kickstarter = True
force_clean_small_kickstarter = True
force_clean_indiegogo = True

# Clean large kickstarter
my_kickstarter_large_file = Path('./Resources/kickstarter_data_clean/ks-projects-large.csv')
if not my_kickstarter_large_file.is_file() or force_clean_large_kickstarter:
    %run ./Resources/Utils/clean_kickstarter_large.ipynb

# Clean small kickstarter
my_kickstarter_small_file = Path('./Resources/kickstarter_data_clean/ks-projects-small.csv')
if not my_kickstarter_small_file.is_file() or force_clean_small_kickstarter:
    %run ./Resources/Utils/clean_kickstarter_small.ipynb

# Clean indiegogo kickstarter
my_indiegogo_file = Path('./Resources/indiegogo_data_clean/indiegogo-projects.csv')
if not my_indiegogo_file.is_file() or force_clean_indiegogo:
    %run ./Resources/Utils/clean_indiegogo.ipynb

## Enable Mapbox API access token

In [ ]:
# Load the .env file into the notebook
load_dotenv()

# Read in your MAPBOX_API_KEY
mapbox_api_access_token = os.getenv('MAPBOX_API_ACCESS_TOKEN')

# Confirm the availability of your Mapbox API access token by checking its type
display(type(mapbox_api_access_token))

# Set your Mapbox API access token
px.set_mapbox_access_token(mapbox_api_access_token)

## Read kickstarter large

In [ ]:
# Import small kickstarter most backed dataset
kickstarter_large_clean_df = pd.read_csv(
    Path('./Resources/kickstarter_data_clean/ks-projects-large.csv')
)

# Fix dates
kickstarter_large_clean_df['launched'] = pd.to_datetime(kickstarter_large_clean_df['launched'])
kickstarter_large_clean_df['deadline'] = pd.to_datetime(kickstarter_large_clean_df['deadline'])

# View head
kickstarter_large_clean_df = kickstarter_large_clean_df.sort_values('launched')
display(kickstarter_large_clean_df.head(2))
display(kickstarter_large_clean_df.tail(2))

## Read kickstarter small

In [ ]:
# Import small kickstarter most backed dataset
kickstarter_small_clean_df = pd.read_csv(
    Path('./Resources/kickstarter_data_clean/ks-projects-small.csv')
)

# View head
kickstarter_small_clean_df.head(2)

## Read indiegogo

In [ ]:
# Import indiegogo dataset
indiegogo_clean_df = pd.read_csv(
    Path('./Resources/indiegogo_data_clean/indiegogo-projects.csv')
)

# Fix dates
indiegogo_clean_df['launched'] = pd.to_datetime(indiegogo_clean_df['launched'])
indiegogo_clean_df['deadline'] = pd.to_datetime(indiegogo_clean_df['deadline'])

# View head
indiegogo_clean_df = indiegogo_clean_df.sort_values('launched')
display(indiegogo_clean_df.head(2))
display(indiegogo_clean_df.tail(2))


## Add SQL database and simple functions to load and access tables in that database

In [ ]:
# Establishes Database Connection with a temporary SQL db (we can update to give it a name later)
database_connection_string = "sqlite:///crowdfunding_db"

engine = sqlalchemy.create_engine(database_connection_string)

# Function to load table into DB
# data is the dataframe we want to save, 
# table name is the name of the new table (as a string value), 
# and engine is the engine input established earlier
def new_table(data, table_name):
    data.to_sql(f"{table_name}", engine, index=True, if_exists="replace")


# Lets us load the table of our choice from the database, just set the function equal to a new dataframe variable and run 
# must set the table name as a string value
def load_full_table(table_name):
    new_df = pd.read_sql_table(f"{table_name}", con=engine )
    return new_df

new_table(kickstarter_large_clean_df, "kickstarter_large")
new_table(kickstarter_small_clean_df, "kickstarter_small")
new_table(indiegogo_clean_df, "indie_gogo")


## Add SQL query to access main_category and mean of duration of projects for Kickstarter and Indiegogo.

In [ ]:
# Create a SQL query to get the main_category and duration of the Kickstarter Large dataframe.
query_ks = """
SELECT main_category, duration
FROM kickstarter_large
"""
query_indiegogo = """
SELECT main_category, duration
FROM indie_gogo
"""

# This will let us read the query we applied earlier to create a dataframe.
ks_large_duration_dataframe = pd.read_sql_query(
    query_ks, 
    con= engine)
display(ks_large_duration_dataframe)

indiegogo_duration_dataframe = pd.read_sql_query(
    query_indiegogo, 
    con= engine)
display(indiegogo_duration_dataframe)

In [ ]:
# Create a SQL query to get the main_category and 
# duration of the Kickstarter Large dataframe and 
# group them by main_category and get its average duration days.
query="""SELECT
   main_category, AVG(duration) AS average_duration_days
FROM
  kickstarter_large
GROUP BY
  main_category
ORDER BY 
    AVG(duration) DESC,
    main_category DESC;
"""
# This will let us read the query we applied earlier to create a dataframe.
ks_large_groupby_maincategory_df = pd.read_sql_query(
    query, 
    con= engine)
ks_large_groupby_maincategory_df


In [ ]:
# This will create a bar plot to visualize the average duration of 
# the projects per main category in Kickstarter.

ks_lrg_duration_hvplot = ks_large_groupby_maincategory_df.hvplot(kind='bar',
                                            x='main_category',
                                            ylim=(30.0,35.0),
                                            rot=60,
                                            title="Kickstarter's Average Duration Days of Each Main Category",
                                            hover_color= 'orange',
                                            color = 'green'
                                            )
ks_lrg_duration_hvplot

In [ ]:
# Create a SQL query to get the main_category and 
# duration of the Indiegogo  dataframe and 
# group them by main_category and get its average duration days.
query="""SELECT
   main_category, AVG(duration) AS average_duration_days
FROM
  indie_gogo
GROUP BY
  main_category
ORDER BY 
    AVG(duration) DESC,
    main_category DESC;
"""
# This will let us read the query we applied earlier to create a dataframe.
indiegogo_groupby_maincategory_df = pd.read_sql_query(
    query, 
    con= engine)
indiegogo_groupby_maincategory_df

In [ ]:
# This will create a bar plot to visualize the average duration of the
# projects per main category in Indiegogo.

indiegogo_duration_hvplot = indiegogo_groupby_maincategory_df.hvplot(kind='bar',
                                            x='main_category',
                                            ylim=(20,46.0),
                                            rot=60,
                                            title="Indiegogo's Average Duration Days of Each Main Category",
                                            hover_color= 'orange',
                                            color = 'magenta'
                                            )
indiegogo_duration_hvplot

## Add SQL query to get the countries of Kickstarter and Indiegogo and get total the number of projects per country

In [ ]:
# Create a SQL query to get the total number of projects in Kickstarter Large dataframe per country.
query_ks_country = """
SELECT full_country_names AS Country, COUNT (*) AS Total_number_of_projects, lat, long 
FROM kickstarter_large
GROUP BY full_country_names
ORDER BY
    total_number_of_projects DESC;
"""

# This will let us read the query we applied earlier to create a dataframe.
ks_country_total_df = pd.read_sql_query(
    query_ks_country, 
    con= engine)
ks_country_total_df

In [ ]:
# Create a SQL query to get the total number of projects in Kickstarter Large dataframe per country.
query_indiegogo_country = """
SELECT full_country_names AS Country, COUNT (*) AS Total_number_of_projects, lat, long 
FROM indie_gogo
GROUP BY full_country_names
ORDER BY
    total_number_of_projects DESC;
"""

# This will let us read the query we applied earlier to create a dataframe.
indiegogo_country_total_df = pd.read_sql_query(
    query_indiegogo_country, 
    con= engine)
indiegogo_country_total_df

## Create scatter mapbox plot for Kickstarter and Indiegogo countries and their respective total number of projects.

In [ ]:
# This will create a scatter mapbox plot for Kickstarter and  their respective longitude and latitude plotted in with total number of projects.
ks_mapbox_plot = px.scatter_mapbox(
    ks_country_total_df,
    title='Kickstarter Total Projects per Country',
    lat='lat',
    lon='long',
    size='Total_number_of_projects',
    hover_name='Country',
    size_max=60,
    zoom= 11, center = {"lat": 37.090240, "lon": -95.712891},
    color_discrete_sequence=["green"],
    height=500,
    width=900
   
)
ks_mapbox_plot

In [ ]:
# This will create a scatter mapbox plot for Indiegogo and  their respective longitude and latitude plotted in with total number of projects.
indiegogo_mapbox_plot = px.scatter_mapbox(
    indiegogo_country_total_df,
    title='Indiegogo Total Projects per Country',
    lat='lat',
    lon='long',
    size='Total_number_of_projects',
    hover_name='Country',
    size_max=60,
    zoom= 11, center = {"lat": 37.090240, "lon": -95.712891},
    color_discrete_sequence=["fuchsia"],
    height=500,
    width=900
   
)
indiegogo_mapbox_plot

## SQL query to get the main_category and percent of successful vs failed projects for Kickstarter

In [ ]:
query = """
SELECT main_category, COUNT (*) AS Total_number_of_projects, AVG(funded_percent) AS average_fund_percentage
FROM kickstarter_large
GROUP BY main_category
ORDER BY
    total_number_of_projects DESC;
"""

ks_main_category_total = pd.read_sql_query(
    query, 
    con= engine)
ks_main_category_total

In [ ]:
# Created a for loop statement to get the percent success and percent failed of each main_category for Kickstarter and added it on the dataframe.
if ks_main_category_total.index.name != 'main_category':
    ks_main_category_total.set_index('main_category', inplace=True)

category_list = ks_main_category_total.index
list_of_percent_success = []

for category in category_list:
    query =  f"""
    SELECT main_category, funded_percent, usd_pledged_real, usd_goal_real
    FROM kickstarter_large
    WHERE main_category == "{category}"
    AND usd_pledged_real >= usd_goal_real
    ORDER BY funded_percent DESC
    """

    ks_main_category_success = pd.read_sql_query(
        query, 
        con= engine)
    percent_success_main_category = ks_main_category_success["main_category"].count() / ks_main_category_total.loc[category, 'Total_number_of_projects']
    
    list_of_percent_success.append(percent_success_main_category)
    
ks_main_category_total["percent_success"] = list_of_percent_success
ks_main_category_total["percent_fail"] = 1 - ks_main_category_total["percent_success"]
ks_main_category_total


## SQL query to get the main_category and percent of successful vs failed projects for Indiegogo

In [ ]:
query = """
SELECT main_category, COUNT (*) AS Total_number_of_projects, AVG(funded_percent) AS average_fund_percentage
FROM indie_gogo
GROUP BY main_category
ORDER BY
    total_number_of_projects DESC;
"""

indiegogo_main_category_total = pd.read_sql_query(
    query, 
    con= engine)
indiegogo_main_category_total

In [ ]:
# Created a for loop statement to get the percent success and percent failed of each main_category for Indiegogo and added it on the dataframe.
if indiegogo_main_category_total.index.name != 'main_category':
    indiegogo_main_category_total.set_index('main_category', inplace=True)

category_list = indiegogo_main_category_total.index
list_of_percent_success = []

for category in category_list:
    query =  f"""
    SELECT main_category, funded_percent, usd_pledged_real, usd_goal_real
    FROM indie_gogo
    WHERE main_category == "{category}"
    AND usd_pledged_real >= usd_goal_real
    ORDER BY funded_percent DESC
    """

    indiegogo_main_category_success = pd.read_sql_query(
        query, 
        con= engine)
    percent_success_main_category = indiegogo_main_category_success["main_category"].count() / indiegogo_main_category_total.loc[category, 'Total_number_of_projects']
    
    list_of_percent_success.append(percent_success_main_category)
    

indiegogo_main_category_total["percent_success"] = list_of_percent_success

indiegogo_main_category_total["percent_fail"] = 1 - indiegogo_main_category_total["percent_success"]

display(indiegogo_main_category_total)

## Create new table with success of each category for questionary use

In [ ]:
# Create new table
percent_success_fail_ks_ig_df = pd.merge(ks_main_category_total, indiegogo_main_category_total, left_index=True, right_index=True)
percent_success_fail_ks_ig_df = percent_success_fail_ks_ig_df[['Total_number_of_projects_x', 'percent_success_x', 'Total_number_of_projects_y', 'percent_success_y']]
percent_success_fail_ks_ig_df.rename(columns={
    "Total_number_of_projects_x": "Total_number_of_projects_ks",
    "percent_success_x": "percent_success_ks",
    "Total_number_of_projects_y": "Total_number_of_projects_ig",
    "percent_success_y": "percent_success_ig"},
    inplace=True
)

new_table(percent_success_fail_ks_ig_df, 'percent_success_fail_ks_ig')


## Plot and analyze success vs fail for Kickstarter.

In [ ]:
# Create an hvplot bar to show the success and failed projects of each main category of Kickstarter.
ks_main_category_total.hvplot.bar(stacked= True,
                                  y=['percent_fail', 'percent_success'],
                                  x= 'main_category',
                                  rot= 60,
                                 title='Kickstarter Main Category Success vs Failed Project',
                                  cmap=['pink', 'green']
                                 )

## Plot and analyze success vs fail for Indiegogo.

In [ ]:
# Create an hvplot bar to show the success and failed projects of each main category of Indiegogo.
indiegogo_main_category_total.hvplot.bar(stacked= True,
                                  y=['percent_fail', 'percent_success'],
                                  x= 'main_category',
                                  rot= 60,
                                 title='Indiegogo Main Category Success vs Failed Project',
                                  cmap=['pink', 'green']
                                 )

## General Summary Stats for Indiegogo

In [716]:
# Generate summary stats for Indiegogo
indiegog_data_df = load_full_table('indie_gogo')
indiegogo_summary_df = indiegog_data_df.drop(columns=['ID', 'name', 'index', 'deadline', 'launched', 'state', 'lat', 'long']).describe()

# View head
indiegogo_summary_df

,usd_goal_real,usd_pledged_real,funded_percent,duration,daily_goal,daily_pledged
count,9309.000000,9.309000e+03,9309.000000,9309.000000,9309.00,9309.00
mean,26653.296548,2.334331e+04,0.786834,42.884413,inf,inf
std,50055.500815,1.680333e+05,4.581792,20.841792,NaN,NaN
min,375.037800,1.000000e+00,0.000000,0.000000,0.94,0.02
25%,3200.000000,1.955490e+02,0.020700,30.000000,83.33,4.58
50%,10000.000000,7.220000e+02,0.123900,45.000000,234.38,18.33
75%,25000.000000,2.753383e+03,0.446600,60.000000,681.82,71.96
max,500000.000000,6.051571e+06,207.054000,720.000000,inf,inf


In [718]:
# plot of pledged with success true
indiegogo_clean_success_df = indiegog_data_df[indiegog_data_df['state'] == 1]

indiegogo_clean_success_df = indiegogo_clean_success_df[indiegogo_clean_success_df['usd_pledged_real'] < 5000000]

display(indiegogo_clean_success_df.hvplot.box(y="usd_pledged_real", by="main_category", ylabel='USD Pledged Real', xlabel='Main Category',title="Pledged by Main Category", height=500, width=1000, yformatter='%.0f'))

:BoxWhisker   [main_category]   (usd_pledged_real)

## General Summary Stats for Kickstarter

In [719]:
# Generate summary stats for Kickstarter
kickstarter_data_df = load_full_table('kickstarter_large')

kickstarter_summary_df = kickstarter_data_df.drop(columns=['ID', 'name', 'index', 'deadline', 'launched', 'state', 'lat', 'long', 'avg_backer_per_day', 'pledged_per_person']).describe()

# View head
kickstarter_summary_df

,usd_goal_real,usd_pledged_real,funded_percent,duration,daily_goal,daily_pledged
count,2.274990e+05,2.274990e+05,227499.000000,227499.000000,227499.00,227475.000
mean,5.019878e+04,9.799910e+03,4.010140,32.958479,inf,inf
std,1.262009e+06,8.555011e+04,334.397301,12.800626,NaN,NaN
min,1.500000e-01,0.000000e+00,0.000000,0.000000,0.00,0.000
25%,2.000000e+03,4.625000e+01,0.006400,29.000000,68.97,1.380
50%,5.311080e+03,8.200000e+02,0.205300,29.000000,175.21,27.590
75%,1.594354e+04,4.733000e+03,1.087500,36.000000,517.24,159.495
max,1.663614e+08,1.239314e+07,104277.890000,91.000000,inf,inf


:BoxWhisker   [main_category]   (usd_pledged_real)

## Create a WordCloud to show the main categories for Kickstarter

In [ ]:
# Create a WordCloud to show the main categories for Kickstarter.

# Create a list of word
ks_cat = kickstarter_large_clean_df['category']
ks_string = " ".join(ks_cat)
mask = np.array(Image.open('./Resources/Images/K.png'))

# Create the wordcloud object
wordcloud = WordCloud(width=50, height=50, margin=0, max_words=200, max_font_size= 100, colormap='summer', random_state= 10, mask=mask).generate(ks_string)

# Display the generated image:
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()


## Create a WordCloud to show the main categories for Indiegogo

In [ ]:
# Create a WordCloud to show the main categories for Indiegogo.

# Create a list of word
indie_cat = indiegogo_clean_df['category']
indie_string = " ".join(indie_cat)

mask = np.array(Image.open('./Resources/Images/I.png'))

# Create the wordcloud object
wordcloud = WordCloud(width=100, height=100, margin=0, max_words=200, max_font_size=100, colormap='spring', random_state=6, mask=mask).generate(indie_string)

# Display the generated image:
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()



## Box Plots Comparing Indiegogo and Kickstarter

In [ ]:
# Load each dataset for a boxplot
indiegogo_box_df = load_full_table('indie_gogo').drop(columns=['ID', 'name', 'index', 'deadline', 'launched', 'state', 'lat', 'long'])

kickstarter_box_df = load_full_table('kickstarter_large').drop(columns=['ID', 'name', 'index', 'deadline', 'launched', 'state', 'lat', 'long', 'avg_backer_per_day', 'pledged_per_person'])

# Adds a source column in each df with their crowdfunding source location
indiegogo_box_df["source"] = "Indiegogo"
kickstarter_box_df["source"] = "Kickstarter"

# Concats the two dfs to make a comparative boxplot easy to do
all_box_df = pd.concat([indiegogo_box_df, kickstarter_box_df], join="inner", axis=0)

# creates a function to filter out outlier data (for plotting purposes and combine the two dfs with the filtered data)
# column variable must be a string value
def box_plot_filter(df_1, df_2, column):

    # Establishes the 1st and 3rd quartile to calculate the Interquartile range (IQR)
    Q1 = df_1[f"{column}"].quantile(0.25)
    Q3 = df_1[f"{column}"].quantile(0.75)
    IQR = Q3 - Q1

    # Uses the IQR to create a filter to filter out outliers
    filter_1 = (df_1[f"{column}"] >= Q1 - 1.5 * IQR) & (df_1[f"{column}"] <= Q3 + 1.5 *IQR)
    
    # establishes a quartile range and filter for the second dataset
    Q1_2 = df_2[f"{column}"].quantile(0.25)
    Q3_2 = df_2[f"{column}"].quantile(0.75)
    IQR_2 = Q3_2 - Q1_2

    filter_2 = (df_2[f"{column}"] >= Q1_2 - 1.5 * IQR_2) & (df_2[f"{column}"] <= Q3_2 + 1.5 *IQR_2)


    # Filters the datasets with the appropriate filter and then concats them together
    new_df_1 = df_1.loc[filter_1]
    new_df_2 = df_2.loc[filter_2] 

    result_df = pd.concat([new_df_1, new_df_2], join="inner", axis=0)

    return result_df

# Runs the filter function on the 4 subsets of data to plot
goal_df = box_plot_filter(indiegogo_box_df, kickstarter_box_df, "usd_goal_real")
pledged_df = box_plot_filter(indiegogo_box_df, kickstarter_box_df, "usd_pledged_real")
funded_df = box_plot_filter(indiegogo_box_df, kickstarter_box_df, "funded_percent")
duration_df = box_plot_filter(indiegogo_box_df, kickstarter_box_df, "duration")

In [ ]:
# Boxplot comparing the two crowdfunding sources in each of the 4 categories

display(goal_df.hvplot.box(y="usd_goal_real", by="source", title="Goal Funding in USD", height=400, width=500, yformatter='%.0f'))

display(pledged_df.hvplot.box(y="usd_pledged_real", by="source", title="Pledged Funding in USD", height=400, width=500, yformatter='%.0f'))

display(funded_df.hvplot.box(y="funded_percent", by="source", title="Percent funded", height=400, width=500, yformatter='%.2f'))

display(duration_df.hvplot.box(y="duration", by="source", title="Duration of crowd funding", height=400, width=500, yformatter='%.0f'))